In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np

In [5]:
STORM_NUMBER = 12
TITLE = "16-17 January 2021"
SUBTITLE = "8 AM 18 January 2021"
sts = utc(2021, 1, 17, 10)
ets = utc(2021, 12, 30, 23, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 40, 'z': 'yes', 'f': 'linear', 'v': 'snow'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [6]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2020-2021 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df_useme_plot)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'],
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [15]:
cull = ['IA-WB-17', 'IA-DV-2', 'LYNW3', 'SBNW3']
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["state"] == "MN"].index, USEME] = False
hardcode = [
]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=2, upper=4)
res = mp.postprocess(filename='210118.png')
mp.close()

    state  wfo  val         lon        lat                               geo  \
146    NH  GYX  3.9  -71.250000  44.260000    POINT (2229769.547 320491.966)   
147    WV  PBZ  3.9  -79.430000  39.100000   POINT (1754911.568 -436411.614)   
148    PA  PBZ  3.8  -79.800000  40.770000   POINT (1681116.601 -262428.805)   
149    MT  MSO  3.7 -114.200000  48.360000   POINT (-1042632.923 466740.043)   
150    OH  CLE  3.5  -81.320000  41.430000   POINT (1541461.766 -220432.561)   
..    ...  ...  ...         ...        ...                               ...   
449    PA  CTP  2.0  -78.600000  41.566700   POINT (1756132.468 -150604.995)   
450    AK  AJK  2.0 -130.020679  55.912184  POINT (-1819166.543 1573054.661)   
451    NY  BGM  2.0  -75.519700  42.511700     POINT (1969740.443 22194.395)   
452    WA  SEW  2.0 -121.742200  46.786100   POINT (-1630257.464 421714.888)   
453    KY  RLX  2.0  -83.018100  38.494400   POINT (1467630.198 -572488.406)   

     used_for_analysis  nwsli  plotme s